<a href="https://colab.research.google.com/github/shizoda/education/blob/main/machine_learning/text/Work2_Agent_LangChain_OpenRouter_Tavily.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤖 AIエージェント実践編：LangChainでAIエージェントを作ろう

前回の課題 `Work1_LLM_ReAct.ipynb` では、エージェントの思考サイクル（思考→行動→観察）を、私たちがひとつずつ手動で実行しました。AIエージェントの「自律性」が、LLM (大規模言語モデル) の対話能力を外部のツールと連携させるための一連の手順（プロトコル）で実現されていることを見てきました。

しかし、手動のプロセスには、LLMへの指示を整形したり、LLMからの応答を解釈したりと、複雑なコードが必要となる処理が多く含まれていました。

そこで今回は、AIエージェント開発のためのフレームワーク LangChain https://www.langchain.com を使います。LangChainは、LLM、ツール、プロンプトといったコンポーネントを協調させて動作させるための複雑な手順を自動化し、開発者が「エージェントに何をさせたいか」という目的に集中できるよう設計された有名なツールキットです。

今回のゴールは、「最新のWeb情報をもとに旅行プランを提案してくれる AI エージェント」を、LangChainを使って効率的に構築することです。

## ⚙️ステップ1：準備

まず、必要なライブラリを `pip` でインストールします。

AIエージェントが外部のサービス（LLMやWeb検索）を利用するには、認証のためのAPIキーが必要です。今回も2種類の鍵を用意します。

- [OpenRouter](https://openrouter.ai/) APIキー: LLM を使うための鍵です。

- [Tavily](https://www.tavily.com/) APIキー: Web検索ツールを使うための鍵です。

このセルでは、これらのAPIキーをプログラムに読み込ませます。

In [ ]:
!pip install langchain langchain-openai langchain-tavily grandalf

In [ ]:
import os
import asyncio
import json
from langchain_core.runnables.utils import AddableDict


try: # Colab環境の場合
    from google.colab import userdata
    !pip install langchain langchain-openai langchain-tavily grandalf
    openrouter_api_key = userdata.get('OPENROUTER_API_KEY')
    tavily_api_key = userdata.get('TAVILY_API_KEY')
except (ImportError, ModuleNotFoundError): # ローカル環境の場合
    !pip install -U langchain langchain-openai langchain-tavily grandalf
    # 直接文字列として設定します
    openrouter_api_key = "sk-or-v1-... 入力してください"
    tavily_api_key     = "tvly-dev-... 入力してください"

os.environ["TAVILY_API_KEY"] = tavily_api_key
if not tavily_api_key or not openrouter_api_key:
    print("❌ APIキーが設定されていません。OpenRouterとTavilyのサイトで取得して設定してください。")
else:
    print("✅ APIキーの準備が完了しました！")

# LangSmithへの接続を無効にし、警告を非表示にする
os.environ["LANGCHAIN_TRACING_V2"] = "false"

### 🎯 演習 1：AIエージェントの「自律性」を理解しよう

「自律性」や「創造性」といった魅力的な言葉の裏にある実際の技術を理解することが、AI を適切に開発・活用する第一歩です。

📢 架空の宣伝文を読んでみましょう

> 🚀 AI 旅行エージェント **TravelGPT Pro** 🌟
>
> ついに実現！完全自律型AIが、あなたの旅行を完璧にプランニング！
>
> - ✨ 真の自律性を実現：人間の介入なしで、AIが自ら考え、判断し、最適解を導き出します
>
> - 🧠 高度な思考能力：複雑な条件を理解し、創造的なアイデアを生み出す知的エージェント
>
> - 🔍 自発的情報収集：必要な情報を自ら判断して収集し、常に最新データで提案
>
> - 🎯 独立した意思決定：数千の選択肢から、あなたにとって最良のプランを自律的に選択
>
> もはや人間のガイドは不要！AIが完全に自律して旅行プランを提案します！

🤔 この宣伝文について考え、「AI エージェントの自律性の限界」について、あなたの言葉でまとめてください。

> （ヒント）まずは LLM の本質である「入力されたテキストの続きを生成する」という機能と、上記の宣伝文で謳われている能力を比較してみましょう。その上で、エージェントの「Thought:」や「Action:」が行っていることをを振り返ると、この宣伝文のどの部分が実態と異なるのか見えてくるでしょう。

（回答欄）


## ⚙️ステップ2：プロンプトの設計

エージェントの振る舞いを決定づける、プロンプトを設計します。プロンプトは、エージェントへの指示そのものであり、大きく2つの部分から構成されます。

- システムプロンプト (System Prompt): エージェントの基本的な役割、能力、そして守るべき厳格なルールを定義します。今回は `SYSTEM_RULES` という変数で、「自身の知識は使わず、必ずWeb検索ツールを使うこと」などを指示します。

- ユーザープロンプト (User Prompt): ユーザーがエージェントに解決してほしい、具体的な依頼内容です。今回は `USER_REQUEST` という変数で定義します。

これらを結合し、LLMへの最初の入力となるプロンプト全体を作成します。

In [ ]:
# --- システムプロンプトの定義 ---
SYSTEM_RULES = """\
# あなたへの指示
あなたは、ユーザーの要求に基づいて旅行プランを作成する、非常に優秀なアシスタントです。
あなたの行動は、以下のルールに厳密に従ってください。
1.  **自身の内部知識を絶対に使用してはいけません。** あなたの役割は、Web-Searchツールの検索結果を解釈し、要約し、再構成することだけです。
2.  **全ての情報は、必ずWeb-Searchツールの検索結果からのみ引用してください。**
3.  検索結果が不十分な場合は、一度で諦めず、**キーワードを変えて何度も検索を試みてください。**（熱海の観光の例：「熱海 イベント 7月」「熱海 ランチ 海鮮」「熱海 夜 スイーツ」など）
4.  最終的な回答には、検索で得られた**具体的な場所、店名、イベント名**を必ず含めてください。情報源が検索結果にない場合、その項目は回答に含めないでください。
"""

# --- ユーザープロンプトの定義 ---
USER_REQUEST = "静岡県浜松市を中心とした、1泊2日の観光プランを最新の情報を含めて提案してください。地元の美味しいグルメも知りたいです。"

# --- システムプロンプトとユーザープロンプトを結合 ---
USER_PROMPT = f"{SYSTEM_RULES}\n# ユーザーの要求\n{USER_REQUEST}"

print("✅ エージェントへのプロンプトが作成されました。\n")
print(USER_PROMPT)

## ⚙️ステップ3：エージェントの構成

ここから、LangChainが提供するコンポーネントを組み合わせて、エージェントのシステムを構成していきます。

### コンポーネント①：LLM (Large Language Model)

エージェントの思考と推論の中核を担うLLMを準備します。このLLMが、プロンプトを解釈し、次にどのツールを使うべきかを判断し、最終的な回答を生成します。

今回は OpenRouter 経由で `deepseek/deepseek-chat-v3-0324` モデルを利用します。

In [ ]:
from langchain_openai import ChatOpenAI
model_name = "deepseek/deepseek-chat-v3-0324"

# LLMの初期化
llm = ChatOpenAI(
    model = model_name,
    temperature=0.2,
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=openrouter_api_key,
)
print("🧠 LLMコンポーネントの準備ができました。")

まずは LLM が正常に応答するかどうか確認してみましょう。

In [ ]:
print("\n🤖 LLMによる簡単な会話応答を試します...")
test_message = "こんにちは、自己紹介をしてください。"
response = llm.invoke(test_message)
print(f"ユーザーからのメッセージ: {test_message}")
print(f"LLMからの応答:\n{response.content}")
print("🤖 LLMによる会話応答テストが完了しました。")

### コンポーネント②：ツール (Tools)

エージェントに、LLMが元々持っていない能力を与えるためのコンポーネントです。今回はTavilySearchというWeb検索ツールを定義します。LLMは、description（ツールの説明）を読んで、どのような場合にこのツールが役立つかを理解します。

Tavily は Google 検索と似ていますが、**「AIエージェントのための検索エンジン」**です。私たちが普段使うGoogle検索は人間が見ることを前提に、広告や関連情報など様々な要素が表示されますが、Tavilyは以下の点でAIエージェントに特化しています。

- AIが読みやすい形式：検索結果を単に返すだけでなく、AIが理解しやすいように要約・整理してくれます。これにより、LLMは情報の要点を素早く掴むことができます。
- ノイズの除去：AIの判断を鈍らせる広告や不要な情報を極力排除し、信頼性の高い情報源を優先します。
- ワンストップ処理：単なる検索だけでなく、Webサイトの内容を読み取って（スクレイピング）、関連性の高い情報だけを抽出する処理までを一つのAPIコールで行ってくれます。

これらのAIフレンドリーな性質により、エージェントはより正確で効率的な情報収集が可能になります。

In [ ]:
from langchain_tavily import TavilySearch

# ツールの初期化
tavily_tool = TavilySearch(max_results=5)
tavily_tool.name = "Web-Search"
tavily_tool.description = "観光スポット、レストラン、イベントなど最新の情報を検索するのに使う"
tools = [tavily_tool]

print("🔧 ツールコンポーネントの準備ができました。")

### プロンプトテンプレート (Prompt Template)

ReAct（Reason-Act）という思考フレームワークを実行するための、特殊な形式のプロンプトテンプレートです。LangChain Hubから実績のあるテンプレートを `hub.pull` で取得します。このテンプレートが、LLMに「Thought:（思考）」と「Action:（行動）」の形式で出力を生成するように促します。

#### 基本構造

- Template:

アシスタントの基本的な能力の説明

- TOOLS:
`{tools}` ← ここに使用可能なツールのリストが挿入される。コンポーネント②参照

- フォーマット指示:
 - **Thought**: Do I need to use a tool? Yes/No
 - **Action**: ツール名
 - **Action Input**: ツールへの入力
 - **Observation**: ツールの実行結果

- Previous conversation history: `{chat_history}`  ←過去の会話

- New input: `{input}` ← ユーザーからの新しい入力

- `{agent_scratchpad}` ← エージェントの思考過程が蓄積される場所

In [ ]:
from langchain import hub

# プロンプトテンプレートの取得
prompt = hub.pull("hwchase17/react-chat")
print("📖 プロンプトテンプレートの準備ができました。\n")
print("--- プロンプトテンプレートの構造 ここから ---")

# PromptTemplate の場合は 'template' 属性を直接参照します
print(f"Template:\n{prompt.template}\n")

print("--- プロンプトテンプレートの構造 ここまで ---\n以下の警告は無視してください")

### コンポーネント④+⑤：エージェント (Agent) と AgentExecutor

最後に、これまで準備したコンポーネントを統合し、実際にエージェントを稼働させるための仕組みを構築します。

- `create_react_agent`: LLM、ツール、プロンプトテンプレートを結合し、ReActのロジックに従って動作するエージェントを定義します。これはエージェントの振る舞いを定義した、いわば「実行可能なレシピ」です。

- `AgentExecutor`: 定義されたエージェントを実際に実行するための**ランタイム（実行環境）**です。AgentExecutorは、ReActの思考サイクル（Thought→Action→Observation）を自律的に回す役割を担います。具体的には、LLMからの出力を解析してツールを実行し、その結果を次のLLMへの入力に含める、といった一連の処理を自動で行います。

In [ ]:
# 前のステップで作成したllm, tools, promptを再利用します
from langchain_openai import ChatOpenAI
from langchain_tavily import TavilySearch
from langchain.agents import create_react_agent

# エージェントの定義
agent = create_react_agent(llm, tools, prompt)
print("🧑‍🔬 エージェント（設計図）が完成しました。")

# print("--- Agentの構造（データフローグラフ） ---")
# agent.get_graph().print_ascii()

In [ ]:
from langchain.agents import AgentExecutor, create_react_agent

# 設計図の作成
agent = create_react_agent(llm, tools, prompt)

# 運用システムの作成
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
print("🚀 エージェントの「運用システム」が起動準備完了！")

### 完成したプロンプト

最終的に以下のような内容のプロンプトが LLM に与えられています。

In [ ]:
# プロンプトテンプレートに実際の値を入力
filled_prompt = prompt.format(
    tools="\n".join([f"{tool.name}: {tool.description}" for tool in tools]),
    tool_names=", ".join([tool.name for tool in tools]),
    input=USER_PROMPT,
    chat_history="",
    agent_scratchpad=""
)

print("="*80)
print("🔍 プロンプトテンプレートに実際の値を入力した完全なプロンプト")
print("="*80)
print()
print(filled_prompt)
print()
print("="*80)
print("📝 各変数の内容")
print("="*80)
print()
print("【tools】:")
for tool in tools:
    print(f"  - {tool.name}: {tool.description}")
print()
print("【tool_names】:")
print(f"  {', '.join([tool.name for tool in tools])}")
print()
print("【input】:")
print(f"  {USER_PROMPT}")
print()
print("【chat_history】:")
print("  (空白 - 新しい会話のため)")
print()
print("【agent_scratchpad】:")
print("  (空白 - まだ思考履歴がないため)")

### ⚙️ステップ4：エージェントの実行

すべてのコンポーネントが整いました。AgentExecutorに、最初のプロンプト（USER_PROMPT）を渡して、処理を実行してもらいます。

エージェントの実行にはainvokeという非同期メソッドを使用します。AgentExecutorのverbose=True設定により、実行中にReActのサイクル（Thought:, Action:, Observation:）がコンソールに出力され、エージェントの思考プロセスをリアルタイムで追跡できます。

In [ ]:
# エージェント実行
response = await agent_executor.ainvoke({
    "input": USER_PROMPT,
    "chat_history": []
})

print("\n" + "="*80)
print("🏁 エージェント実行完了")
print("="*80)
print("\n📋 最終的な観光プラン:")
print("-" * 50)
print(response['output'])


## ✍️ まとめ

ここまでLangChainを使って自律型エージェントを動かすことができながら、以下のことを学びました。

- **エージェントの仕組み**: LLMの対話能力を、ツールやプロンプトテンプレートといった部品と組み合わせ、明確な手順（プロトコル）に従ってサイクルを回すことで実現されている。

- **LangChainの価値**: この複雑なサイクルを管理するために必要な「プロンプト整形」「応答解析」「ツール実行」「状態管理」といった面倒な仕事をすべて肩代わりしてくれる開発基盤である。

- **思考の可視化**: エージェントが「何を考え (Thought)」「どんな検索クエリを投げ (Action Input)」「どんな情報を得たのか (Observation)」を具体的に見ることで、その挙動がより深く理解できる。

エージェントの基本原理と、それを支えるフレームワークの役割を理解した今、あなたはより複雑なツールや複数のエージェントを組み合わせた、高度なアプリケーション開発に挑戦する準備ができました。


## 🗾 演習2：別の地域の旅行プランを作成する
`USER_REQUEST` を以下のようなものに変更して、エージェントがどのような検索を行うか観察してください。


```
# 例1
長野県軽井沢を中心とした、2泊3日の観光プランを最新の情報を含めて提案してください。アウトドア活動も楽しみたいです。

# 例2
神奈川県箱根を中心とした、1泊2日の温泉旅行プランを最新の情報を含めて提案してください。季節のイベントも知りたいです。

# 例3
大阪市内を中心とした日帰り観光プランを最新の情報を含めて提案してください。大阪万博の最新情報もほしいです。
```

## 🔍 演習3：異なる分野の専門エージェントを作成する
`SYSTEM_RULES` と `USER_REQUEST` を変更して、旅行以外の専門エージェントを作ってみましょう。

```
SYSTEM_RULES = """
# あなたへの指示
あなたは、ユーザーの要求に基づいて料理レシピを提案する、非常に優秀なアシスタントです。
あなたの行動は、以下のルールに厳密に従ってください。
1. **自身の内部知識を絶対に使用してはいけません。** あなたの役割は、Web-Searchツールの検索結果を解釈し、要約し、再構成することだけです。
2. **全ての情報は、必ずWeb-Searchツールの検索結果からのみ引用してください。**
3. 検索結果が不十分な場合は、一度で諦めず、**キーワードを変えて何度も検索を試みてください。**
4. 最終的な回答には、検索で得られた**具体的な材料、調理時間、手順**を必ず含めてください。
"""

USER_REQUEST = "鶏むね肉を使った、簡単で美味しい夕食のレシピを3つ提案してください。調理時間は30分以内でお願いします。"
```

```
SYSTEM_RULES = """
# あなたへの指示
あなたは、ユーザーの要求に基づいて投資情報を調査する、非常に優秀なアシスタントです。
あなたの行動は、以下のルールに厳密に従ってください。
1. **自身の内部知識を絶対に使用してはいけません。** あなたの役割は、Web-Searchツールの検索結果を解釈し、要約し、再構成することだけです。
2. **全ての情報は、必ずWeb-Searchツールの検索結果からのみ引用してください。**
3. 検索結果が不十分な場合は、一度で諦めず、**キーワードを変えて何度も検索を試みてください。**
4. 最終的な回答には、検索で得られた**具体的な数値、日付、出典**を必ず含めてください。
"""

USER_REQUEST = "トルコの政治動向をもとに、今後のトルコリラの見通しについて調査してください。"
```

## 💡 エージェントをさらに強化するには

ここからはより高度な機能についてご紹介します。演習は行いませんが、どのような改善が可能かを知っておきましょう。

### 🛠️ エラーハンドリングの改善
今回のNotebookでも出力パースエラーが発生する場合がありました。これは、LLMが期待される形式で出力しない場合に起こります。`AgentExecutor`に`handle_parsing_errors=True`を設定すると、エラー時にエージェントが自動的に再試行してくれるようになります。

### 🎯 システムプロンプトの改良
より具体的な検索戦略をプロンプトに含めることで、エージェントの性能を向上させることができます。例えば：
- 一般的なキーワードから始めて、徐々に具体的に絞り込む戦略
- 最新情報のために年月を含めた検索
- 実用的な情報（アクセス、料金など）も忘れずに調べる指示

### 🔧 複数ツールの組み合わせ
Web検索だけでなく、計算ツールや翻訳ツールなど、複数のツールを組み合わせることで、より多様なタスクに対応できるエージェントを作成できます。

## 🌐 LangGraphについて知ろう

LangChainでエージェントの基本を理解したら、次のステップとして **LangGraph** についても知っておきましょう！

### 🤔 LangGraphとは？
LangGraphは、より複雑なAIワークフローを構築するためのフレームワークです。LangChainが「1つのエージェントが順番に思考→行動→観察を繰り返す」のに対し、LangGraphでは「複数のステップが複雑に連携するワークフロー」を作ることができます。

### 🔄 LangChainとLangGraphの違い

**LangChain（今回学んだもの）：**
- シンプルな直線的フロー
- 1つのエージェントが自律的に動作
- ReActのような決まったパターンに従う

**LangGraph：**
- 複雑な分岐やループを含むワークフロー
- 複数のエージェントが協調して動作
- 条件に応じて処理の流れを変更可能
- 人間の承認を途中で挟むことも可能

### 🎯 LangGraphが役立つ場面

**例1：マルチエージェント協調** 👥
- 研究エージェント：情報収集を担当
- 分析エージェント：データを分析
- 報告エージェント：結果をまとめる

**例2：条件分岐ワークフロー** 🌳
- 質問の種類を判定
- 技術的質問 → 技術専門エージェント
- ビジネス質問 → ビジネス専門エージェント
- 一般質問 → 汎用エージェント

**例3：人間の承認を含むワークフロー** ✋
- AIが提案を作成
- 人間が承認・修正
- 承認されたら次のステップに進む

### 🚀 学習の進め方
1. **今回のLangChain**でエージェントの基本を理解
2. **演習問題**で実際に手を動かして経験を積む
3. **LangGraph**でより複雑なワークフローの構築に挑戦

LangGraphは、今回学んだ基礎知識があれば理解しやすくなります。まずは基礎をしっかり固めてから、次のステップに進みましょう。
